In [14]:
fin = open('checkins.dat')
columns = map(lambda x: x.strip(), fin.readline().strip().split('|'))

In [15]:
print columns

['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at']


In [19]:
data = []
p=0
for line in fin:
    if p==0:
        p=1
    else:
        data.append(map(lambda x: x.strip(), line.strip().split('|')))


In [35]:
import pandas as pd
df = pd.DataFrame(data)
df.columns = columns

In [36]:
df.head(5)

,id,user_id,venue_id,latitude,longitude,created_at
0,984315,1764391,5222,,,2012-04-21 17:37:18
1,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
2,984249,2146840,5222,,,2012-04-21 17:42:58
3,984268,2146843,5222,,,2012-04-21 17:42:38
4,984281,2146846,5222,,,2012-04-21 17:39:40


In [39]:
df = df[(df.latitude!='') & (df.longitude!='')]
df.head(5)

,id,user_id,venue_id,latitude,longitude,created_at
1,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
5,984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22
7,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
8,984232,93870,380645,33.4483771,-112.0740373,2012-04-21 17:38:18
9,984483,1030290,955969,32.2217429,-110.926479,2012-04-21 17:58:54


In [41]:
df.shape

(396635, 6)

In [46]:
df.to_csv('checkins.csv', index = None)

In [56]:
df.drop([1021964,1021965],axis=0,inplace=True)

In [58]:
dt.isnull == False

False

In [59]:
dt.shape

(396635, 6)

In [60]:
from sklearn.cluster import MeanShift

In [62]:
data = df[['latitude', 'longitude']]
data = data.applymap(float)
sample_dt = data[:100000]


In [63]:
sample_dt.shape

(100000, 2)

In [64]:
model = MeanShift(bandwidth=0.1)
pred = model.fit_predict(sample_dt)

In [120]:
sample_dt['pred'] = pred

C:\Users\Vladuk\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [121]:
sample_dt.head(10)

,latitude,longitude,pred
1,33.800745,-84.410520,7
5,45.523452,-122.676207,30
7,40.764462,-111.904565,65
8,33.448377,-112.074037,1
9,32.221743,-110.926479,23
10,40.650000,-73.950000,0
11,33.448377,-112.074037,1
13,33.414768,-111.909310,2
16,42.358431,-71.059773,8
17,33.448377,-112.074037,1


In [122]:
n_p = set(pred)
print len(n_p)

3231


In [123]:
cluster_dt_fr = pd.DataFrame(pd.pivot_table(sample_dt, values='longitude', index=pred, aggfunc='count'))

In [124]:
cluster_dt_fr.columns = ['clust_size']
cluster_cent_df = pd.DataFrame(model.cluster_centers_)
cluster_cent_df.columns = ['cent_latitude', 'cent_longitude']

In [125]:
cluster_df = pd.concat([cluster_cent_df,cluster_dt_fr],axis=1)

In [126]:
f_cluster_df = cluster_df[(cluster_df['clust_size']>15)]
f_cluster_df.shape

(592, 3)

In [127]:
f_cluster_df.head()

,cent_latitude,cent_longitude,clust_size
0,40.717716,-73.991835,12506
1,33.449438,-112.002140,4692
2,33.446380,-111.901888,3994
3,41.878244,-87.629843,3363
4,37.688682,-122.409330,3527


In [128]:
def get_distance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5

office_coordinates = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
]

def get_min_distance_to_office(lat, lon):
    min_dist = None
    for (of_lat, of_lon) in office_coordinates:
        dist = get_distance(lat, lon, of_lat, of_lon)
        if (min_dist is None) or (dist < min_dist):
            min_dist = dist
    return min_dist

In [129]:
f_cluster_df['min_distance'] = map(get_min_distance_to_office, f_cluster_df.cent_latitude, f_cluster_df.cent_longitude)

C:\Users\Vladuk\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [130]:
f_cluster_df.head()

,cent_latitude,cent_longitude,clust_size,min_distance
0,40.717716,-73.991835,12506,16.143720
1,33.449438,-112.002140,4692,6.193959
2,33.446380,-111.901888,3994,6.294241
3,41.878244,-87.629843,3363,17.598580
4,37.688682,-122.409330,3527,5.772048


In [131]:
f_cluster_df = pd.DataFrame(f_cluster_df)

In [132]:
f_cluster_df.head()

,cent_latitude,cent_longitude,clust_size,min_distance
0,40.717716,-73.991835,12506,16.143720
1,33.449438,-112.002140,4692,6.193959
2,33.446380,-111.901888,3994,6.294241
3,41.878244,-87.629843,3363,17.598580
4,37.688682,-122.409330,3527,5.772048


In [138]:
f_sort = f_cluster_df.sort_values('min_distance')

In [139]:
f_sort.head()

,cent_latitude,cent_longitude,clust_size,min_distance
420,-33.860630,151.204776,28,0.007835
370,52.372964,4.892317,31,0.009353
419,25.845672,-80.318891,28,0.022674
58,51.502991,-0.125537,254,0.050058
51,33.809878,-118.148924,281,0.070848


In [141]:
answer = str(f_sort.cent_latitude[420]) +', ' + str(f_sort.cent_longitude[420])
print answer

-33.86063042857143, 151.20477592857145
